In [ ]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import contextily as ctx
from shapely.geometry import Point

# ==========================================
# 1. 설정 및 데이터 로드
# ==========================================
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.dpi'] = 100

place_name = "Geumcheon-gu, Seoul, South Korea"
target_crs_metric = "EPSG:5179" 
target_crs_web = "EPSG:3857"

print("1. GTFS 데이터 로드 및 전처리 중...")

# 데이터 로드
stops_df = pd.read_csv('2025-TM-PT-GTFS/202403_GTFS_DataSet/stops.txt')
routes_df = pd.read_csv('2025-TM-PT-GTFS/202403_GTFS_DataSet/routes.txt')

# Stops GeoDataFrame
stops_gdf = gpd.GeoDataFrame(
    stops_df,
    geometry=gpd.points_from_xy(stops_df.stop_lon, stops_df.stop_lat),
    crs="EPSG:4326"
)

# 정류장 분류 함수
def classify_stop(stop_id):
    stop_str = str(stop_id)
    if stop_str.startswith('BS'): return '버스'
    elif stop_str.startswith('RS'): return '지하철/철도'
    else: return '기타'

stops_gdf['mode'] = stops_gdf['stop_id'].apply(classify_stop)

# ==========================================
# 2. 공간 데이터 확보 (OSM)
# ==========================================
print(f"2. {place_name} 공간 데이터 추출 및 분석 중...")

# 시군구 경계
gm_boundary = ox.geocode_to_gdf(place_name).to_crs(target_crs_metric)

# 건물 데이터
# tags = {'building': True}
# try:
#     buildings = ox.features_from_place(place_name, tags=tags)
#     buildings = buildings[buildings.geometry.type.isin(['Polygon', 'MultiPolygon'])]
#     buildings = buildings[['geometry']].to_crs(target_crs_metric)
# except Exception as e:
#     print(f"OSM 데이터 다운로드 실패: {e}")
#     buildings = gpd.GeoDataFrame(columns=['geometry'], crs=target_crs_metric)

buildings = gpd.read_file('data/bd11545.gpkg')

# 정류장 필터링 및 분리
stops_proj = stops_gdf.to_crs(target_crs_metric)
stops_in_gm = gpd.clip(stops_proj, gm_boundary.buffer(100))

# 버스와 지하철 데이터셋 분리
stops_bus = stops_in_gm[stops_in_gm['mode'] == '버스']
stops_subway = stops_in_gm[stops_in_gm['mode'] == '지하철/철도']

print(f"   -> 버스 정류장: {len(stops_bus):,}개, 지하철역: {len(stops_subway):,}개")

# ==========================================
# 3. 접근성 분석 (이중 기준 적용)
# ==========================================
print("3. 이중 기준(버스 250m, 지하철 800m) 접근성 계산 중...")

if not buildings.empty:
    # 3-1. 건물 <-> 버스 거리 계산
    if not stops_bus.empty:
        buildings_bus = gpd.sjoin_nearest(buildings, stops_bus, how='left', distance_col='dist_to_bus')
        # 중복 제거 (건물 하나당 가장 가까운 버스정류장 1개만 남김)
        buildings_bus = buildings_bus[~buildings_bus.index.duplicated(keep='first')]
        # 인덱스 기준 병합을 위해 시리즈 추출
        dist_bus_series = buildings_bus['dist_to_bus']
    else:
        dist_bus_series = pd.Series(9999, index=buildings.index)

    # 3-2. 건물 <-> 지하철 거리 계산
    if not stops_subway.empty:
        buildings_subway = gpd.sjoin_nearest(buildings, stops_subway, how='left', distance_col='dist_to_subway')
        buildings_subway = buildings_subway[~buildings_subway.index.duplicated(keep='first')]
        dist_subway_series = buildings_subway['dist_to_subway']
    else:
        dist_subway_series = pd.Series(9999, index=buildings.index)

    # 3-3. 건물 데이터프레임에 거리 정보 합치기
    buildings_final = buildings.copy()
    buildings_final['dist_to_bus'] = dist_bus_series
    buildings_final['dist_to_subway'] = dist_subway_series

    # 결측치(매칭 안됨) 처리
    buildings_final['dist_to_bus'] = buildings_final['dist_to_bus'].fillna(9999)
    buildings_final['dist_to_subway'] = buildings_final['dist_to_subway'].fillna(9999)

    # 3-4. 복합 기준 적용
    # 버스 250m 이내 OR 지하철 800m 이내면 '양호', 둘 다 아니면 '취약'
    def check_accessibility(row):
        if row['dist_to_bus'] <= 250 or row['dist_to_subway'] <= 800:
            return '양호'
        else:
            return '취약'

    buildings_final['access_group'] = buildings_final.apply(check_accessibility, axis=1)

else:
    print("분석할 건물 데이터가 없습니다.")
    buildings_final = gpd.GeoDataFrame()


# ==========================================
# 4. 지도 시각화
# ==========================================
print("4. 지도 시각화 생성 중...")

if not buildings_final.empty:
    # 웹 좌표계 변환
    buildings_web = buildings_final.to_crs(target_crs_web)
    stops_web = stops_in_gm.to_crs(target_crs_web)
    
    fig, ax = plt.subplots(1, 1, figsize=(15, 15))

    # 데이터 분리
    good_access = buildings_web[buildings_web['access_group'] == '양호']
    bad_access = buildings_web[buildings_web['access_group'] == '취약']

    # (1) 접근성 양호 건물 (회색, 배경)
    if not good_access.empty:
        good_access.plot(ax=ax, color='#95a5a6', alpha=0.3, zorder=1)

    # (2) 접근성 취약 건물 (형광 마젠타, 강조)
    if not bad_access.empty:
        bad_access.plot(ax=ax, color='#FF00FF', alpha=1.0, zorder=2)
    else:
        print("   참고: 설정된 기준(버스 250m, 지하철 800m) 내에서 취약 건물이 없습니다.")

    # (3) 정류장 플롯
    stops_bus_web = stops_web[stops_web['mode'] == '버스']
    stops_subway_web = stops_web[stops_web['mode'] == '지하철/철도']

    if not stops_bus_web.empty:
        stops_bus_web.plot(ax=ax, color='#2ecc71', markersize=5, alpha=0.8, zorder=3)
    if not stops_subway_web.empty:
        stops_subway_web.plot(ax=ax, color='#f1c40f', markersize=40, marker='D', zorder=4)

    # (4) 배경지도 (V-World Midnight)
    vworld_url = "http://xdworld.vworld.kr:8080/2d/midnight/service/{z}/{x}/{y}.png"
    try:
        ctx.add_basemap(ax, source=vworld_url, zoom=14)
    except:
        ctx.add_basemap(ax, source=ctx.providers.CartoDB.DarkMatter)

    # (5) 범례 업데이트 (기준 명시)
    legend_elements = [
        mpatches.Patch(color='#95a5a6', alpha=0.3, label='접근성 양호 (버스≤250m or 지하철≤800m)'),
        mpatches.Patch(color='#FF00FF', alpha=1.0, label='접근성 취약 (기준 초과)'),
        mpatches.Circle((0,0), color='#2ecc71', label='버스 정류장'),
        mpatches.Patch(color='#f1c40f', label='지하철/철도역')
    ]

    ax.set_title(f'서울 금천구 대중교통 사각지대 분석\n(기준: 버스 250m, 지하철 800m)', fontsize=20, color='black', pad=20)
    
    leg = ax.legend(handles=legend_elements, loc='upper right', 
                    facecolor='black', edgecolor='white', fontsize=12)
    for text in leg.get_texts():
        text.set_color("white")

    ax.set_axis_off()
    plt.tight_layout()
    plt.show()
    
    # 결과 요약
    print(f"\n=== 분석 결과 요약 ===")
    print(f"기준: 버스 250m 이내 OR 지하철 800m 이내")
    print(f"총 건물: {len(buildings_final):,}개")
    print(f"취약 건물: {len(bad_access):,}개 ({len(bad_access)/len(buildings_final)*100:.1f}%)")
    
    # 어떤 기준 때문에 양호가 되었는지 비율 확인
    only_bus = len(buildings_final[(buildings_final['dist_to_bus'] <= 250) & (buildings_final['dist_to_subway'] > 800)])
    only_subway = len(buildings_final[(buildings_final['dist_to_bus'] > 250) & (buildings_final['dist_to_subway'] <= 800)])
    both = len(buildings_final[(buildings_final['dist_to_bus'] <= 250) & (buildings_final['dist_to_subway'] <= 800)])
    
    print(f"\n[양호 건물 상세]")
    print(f"- 버스만 접근 가능 (250m 이내): {only_bus:,}개")
    print(f"- 지하철만 접근 가능 (800m 이내): {only_subway:,}개")
    print(f"- 둘 다 접근 가능 (역세권+): {both:,}개")